In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

df = pd.read_csv("/Users/nguyenhuuan/Desktop/Dart_KOREA/Metadata/filtered_2010_2023.csv")
df.shape

In [ ]:
def get_text_menu(report_id):
    with open(f"FirstPage/{report_id}.html", "rb") as f:
        soup = BeautifulSoup(f.read(), "html.parser")

    list_script = soup.find_all("script")
    count = 0
    text = ""
    for s in list_script:
        if "function makeToc()" in s.text:
            count += 1
            text = " ".join(s.text.split())
    assert count == 1
    return text

def get_att(list_button_text):
    list_data = []
    for i in range(len(list_button_text)):
        text = list_button_text[i]

        node_lv = text[0]
        assert int(node_lv) in [1,2,3]
        node_name = "node" + node_lv
        pattern = rf"{node_name}\['(.*?)\";"
        list_att = []
        for att in re.findall(pattern, text):
            list_att.append(att.replace("']","").split(" = "))

        data = {"node_level": node_lv}
        data.update({att[0].strip(): "".join(att[1].split()) for att in list_att})
        list_data.append(data)

    df = pd.DataFrame(list_data)
    for col in df.columns:
        if col == "node_level": continue
        assert df[col].str.startswith('"').all()
        df[col] = df[col].str[1:]

    return df

for report_id in tqdm(df["report_id"]):
    text = get_text_menu(report_id)
    pattern =  r"var node(.*?)cnt\+\+;"
    list_button_text = re.findall(pattern,text)

    df_menu = get_att(list_button_text)
    df_menu.to_csv(f"Menu/{report_id}.csv", index=False)